In [11]:
# Author Emmanuel Sedicol
import cv2
import imutils
import numpy as np
import import_ipynb
import tensorflow as tf
import matplotlib.pyplot as plt
from imutils.video import FileVideoStream
import time

from IPython.display import clear_output
from imutils.video import VideoStream
from imutils.video import FPS
from skimage import measure

import POSITION_EXTRACTION as pex
import MAIN_APP as mn
from YOLO import yolo

DEBUG = False
model = tf.keras.models.load_model("MODEL")

# Ball Detection Test

In [12]:
# transform input frame to the corect structure before being classified my the model
def predict_object(frame):
    model = tf.keras.models.load_model("MODEL")
    CATEGORIES = ['basketball', 'hoop']
    
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (80,80))
    img = img.reshape(-1, 80, 80, 1)
    
    # predict image
    prediction = model.predict([img])
    prediction_label = CATEGORIES[int(prediction[0][0])]
    return(prediction_label)

def opencv_ball_detection(frame, bol):
    detected_object = frame.copy()
    
    # frame filtering and image processing
    blur = cv2.GaussianBlur(frame, (15, 15),0)
    erode = cv2.erode(blur, None, iterations=2)
    dilate = cv2.dilate(erode, None, iterations=2)
    hsv = cv2.cvtColor(dilate, cv2.COLOR_BGR2HSV)
    
    # create mask using lower and upper orange range
    mask = cv2.inRange(hsv, (1, 80, 110), (8,200,175))

    # find contours in the mask and initialize the current
    cnts = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if  len(cnts) > 0:   
        
        # get maximum contour based on its area value
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        
        # check if the radius meets size criteria
        if radius > 5 and radius < 20 :
            
            # Retrive the frame of detected ball
            detected_object = detected_object[int(y - 20):int(y + 20),int(x - 20):int(x + 20)] 

            if bol == True:
                # if detected object is classifies as a basketball
                if predict_object(detected_object) == "basketball" :
                    # Draw the circle and centroid on the frame,
                    cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                    cv2.circle(frame, (int(x), int(y)), 1, (0, 0, 255), 2)

                    # Draw rectangle around the minimum enclosed circle and dipslay label of detected object               
                    cv2.rectangle(frame, (int(x - 20), int(y - 20)), (int(x + 20), int(y + 20)), (255,0,0), 2)
                    cv2.putText(frame, "BASKETBALL", (int(x - 30), int(y - 25)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1 )
            else:
                # if detected object is classifies as a basketball
                if predict_object(detected_object) != "basketball" :
                    # Draw the circle and centroid on the frame,
                    cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                    cv2.circle(frame, (int(x), int(y)), 1, (0, 0, 255), 2)

                    # Draw rectangle around the minimum enclosed circle and dipslay label of detected object
                    cv2.rectangle(frame, (int(x - 20), int(y - 20)), (int(x + 20), int(y + 20)), (255,0,0), 2)
                    cv2.putText(frame, "BASKETBALL", (int(x - 30), int(y - 25)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1 )
            
    return x, y, frame, detected_object

In [13]:
if DEBUG:
    shot_xy_coordinates = []
    path = "VIDEOS/BALL_DETECTION_1.mov"

    fvs = FileVideoStream(path).start()
    time.sleep(1.0)
    fps = FPS().start()

    while fvs.more():
        frame = fvs.read()

        # break out of the application if we have no more frames to read
        if frame is None:
            break

        # resize frame so to make its constant size througout
        frame = cv2.resize(frame,(600, 400))
        detected_object = frame.copy()
        
         # Ball Detection Test 1 (OpenCV)
        x, y, frame, detected_object = opencv_ball_detection(frame, True)
        shot_xy_coordinates.append((x,y))
        
#          # Ball Detection Test 2 (YOLO)
#         target_b = "ball"
#         frame, detected, _, _, _ = yolo.yolo_detection(target_b, frame)
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 6))
        fig.suptitle('Basketball and Player Detection')

        ax1.set_title("Main Image")
        ax2.set_title("Shot Plot")
        ax3.set_title("Detected Object")

        ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # plot at realtime the shot coordinates
        for x in range(0, len(shot_xy_coordinates)):
            ax2.scatter(shot_xy_coordinates[x][0], shot_xy_coordinates[x][1])

        # Plot Limit equals the size of the frame
        ax2.set_xlim(0, 600)
        ax2.set_ylim(0,400)
        ax2.invert_yaxis()
        
        ax3.imshow(cv2.cvtColor(detected_object, cv2.COLOR_BGR2RGB))
        
        plt.show()
        clear_output(wait=True)

    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    fvs.release()

# Player Detection Test
- YOLO Detection
- Background Subtraction Dectection

### YOLO Detection Test

In [14]:
if DEBUG:    
    path = "VIDEOS/BALL_DETECTION_2.mov"

    vs = cv2.VideoCapture(path)
    fps = FPS().start()

    while True:
        # grab the current frame
        _,frame = vs.read()

        # break out of the application if we have no more frames to read
        if frame is None:
            break

        # resize frame so to make its constant size througout
        frame = cv2.resize(frame,(600, 450))
        detected_object = frame.copy()

        # detect person using YOLO
        target = "person"
        yolo.yolo_detection(target, frame)

        # plot results
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        fig.suptitle('Basketball and Player Detection')
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        ax.axis("off")
        plt.show()
        clear_output(wait=True)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    vs.release()

### Detection using Background Subtraction Method

In [15]:

vs = cv2.VideoCapture('VIDEOS/BALL_DETECTION_1.mov') 
fps = FPS().start()

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

if DEBUG:
    while True:
        # grab the current frame
        _, frame = vs.read()

        if frame is None:
            break

        frame = cv2.resize(frame,(600, 400))

        blur = cv2.GaussianBlur(frame, (15, 15),0)
        dilate = cv2.dilate(blur, None, iterations=2)
        fgmask = fgbg.apply(dilate)
        dilate = cv2.dilate(fgmask, kernel)

        image, (cnts, heirarchy) = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(cnts):
            area = cv2.contourArea(contour)

            if area > 10:
                x,y,w,h = cv2.boundingRect(contour)

                x_coordinate = int(x + (w/2))
                y_coordinate = int(y + h)

                cv2.rectangle(frame, (x,y), (x+w, y+h), (90,10,255), 2)
                cv2.putText(frame, "PERSON", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1 )
                cv2.circle(frame, (x_coordinate, y_coordinate), 3, (0, 0, 255), 8)


        fig, (ax1) = plt.subplots(figsize=(24, 8))
        ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        ax1.axis("off")
        plt.show()
        clear_output(wait=True)

        fps.stop()
        print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
        print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
        vs.release()